In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 139 kB 49.8 MB/s 
     |████████████████████████████████| 97 kB 7.5 MB/s 
     |████████████████████████████████| 180 kB 47.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=8f9e6bbc37cf274939dfc4c92aaff9b563987a5d89ef8c938272f4913e46277f
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=e38ae2be43c6060d41c40d67cafbe057f291e77966dd74e615a8e331dda69532
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
# Installing pre-requisite for saving the model
!pip install pyyaml h5py

In [ ]:
# Mount google drive where dataset is stored at
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import os
import numpy as np
import pandas as pd

In [ ]:
# Configuration settings 
configs = {
    'epochs': 20,
    'batch_size': 64, 
    'seed': 7,
    'learning_rate': 1e-3, #0.001
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'optimizer': 'adam',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
    'fc_layer_1_neurons' : 1024,
    'fc_layer_2_neurons' : 512,
    'fc_layer_3_neurons' : 256,
}

# Initialise Wandb
inceptionV3_evaluation = wandb.init(
    name='inceptionV3',
    project='CZ4042_Assignment_2',
    config= configs,
)

config = wandb.config
tf.random.set_seed(config.seed)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Define the google drive path where data is stored
base_path = '/content/drive/MyDrive/CZ4042 Project Assignment 2/Models/'

# Load preprocessed dataset containing image path and gender as dataframe
df = pd.read_csv(base_path + 'processed_path_and_gender.txt', sep='\t')

# Split the dataset 80% train / 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state = config.seed)

# Append the google drive path infront of the image path 
train_df['data_path'] = base_path + train_df['data_path'].astype(str)
test_df['data_path'] = base_path + test_df['data_path'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Initialise ImageDataGenerator 
# Perform rescaling using inceptionV3's preprocess_input method
train_image_generated = ImageDataGenerator(preprocessing_function = preprocess_input)
test_image_generated = ImageDataGenerator(preprocessing_function = preprocess_input)

# Generate the rescaled train images from the dataframe given 
train_ds = train_image_generated.flow_from_dataframe(
    dataframe = train_df,
    x_col ='data_path',
    y_col = 'gender',
    batch_size = config.batch_size,
    seed = config.seed,
    shuffle = True,
    class_mode ='raw',
    target_size = (224,224),
)

# Generate the rescaled test images from the dataframe given 
test_ds = test_image_generated.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'data_path',
    y_col = 'gender',
    batch_size = config.batch_size,
    seed = config.seed,
    shuffle = True,
    class_mode = 'raw',
    target_size = (224,224),
)

Found 9755 validated image filenames.
Found 2439 validated image filenames.


In [ ]:
# Creating the inceptionV3 model
def get_inceptionV3_model():

  seq = tf.keras.models.Sequential()

  seq.add(InceptionV3(include_top=False, pooling = 'avg', weights ='imagenet'))

  seq.add(Flatten())

  seq.add(BatchNormalization())

  seq.add(Dense(config.fc_layer_1_neurons, activation = config.hidden_activation))

  seq.add(BatchNormalization())

  seq.add(Dense(config.fc_layer_2_neurons, activation = config.hidden_activation))

  seq.add(BatchNormalization())

  seq.add(Dense(config.fc_layer_3_neurons, activation = config.hidden_activation))

  seq.add(BatchNormalization())

  seq.add(Dense(1, activation= config.output_activation))
  
  seq.layers[0].trainable = False

  return seq

In [ ]:
model = get_inceptionV3_model()

# Compile the model
model.compile(
    optimizer = config.optimizer,
    loss = config.loss_function,
    metrics = config.metrics,
)

# View summary of model
model.summary()

87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
batch_normalization_94 (Batc (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_95 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________

In [ ]:
# Checkpoint storing path
checkpoint_path = base_path + "Model Checkpoints/" + "inceptionV3_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Model checkpoint to store best weights
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Training the model
train_history = model.fit(
    train_ds,
    epochs= config.epochs,
    use_multiprocessing=True,
    callbacks=[model_checkpoint_callback, WandbCallback()],
    validation_data=test_ds,
)

Epoch 1/20
153/153 [==============================] - 3488s 23s/step - loss: 0.4304 - accuracy: 0.8098 - val_loss: 0.3480 - val_accuracy: 0.8516
Epoch 2/20
153/153 [==============================] - 163s 1s/step - loss: 0.2599 - accuracy: 0.8928 - val_loss: 0.3522 - val_accuracy: 0.8639
Epoch 3/20
153/153 [==============================] - 160s 1s/step - loss: 0.1808 - accuracy: 0.9309 - val_loss: 0.3289 - val_accuracy: 0.8696
Epoch 4/20
153/153 [==============================] - 160s 1s/step - loss: 0.1292 - accuracy: 0.9489 - val_loss: 0.4010 - val_accuracy: 0.8585
Epoch 5/20
153/153 [==============================] - 160s 1s/step - loss: 0.0979 - accuracy: 0.9616 - val_loss: 0.4402 - val_accuracy: 0.8708
Epoch 6/20
153/153 [==============================] - 163s 1s/step - loss: 0.0780 - accuracy: 0.9715 - val_loss: 0.4726 - val_accuracy: 0.8590
Epoch 7/20
153/153 [==============================] - 164s 1s/step - loss: 0.0735 - accuracy: 0.9709 - val_loss: 0.4609 - val_accuracy: 0.87

In [ ]:
# Save the inceptionV3 model into a .h5 file
model.save(base_path + 'Saved Models and Weights/' + 'my_inceptionV3_model.h5')

In [ ]:
# Evaluates the inceptionV3 model and prints its accuracy
loss, acc = model.evaluate(
    test_ds,
    callbacks=[WandbCallback()],
)

print("Val_accuracy", acc)

39/39 [==============================] - 30s 765ms/step - loss: 0.5972 - accuracy: 0.8766
Val_accuracy 0.876588761806488


In [ ]:
inceptionV3_evaluation.finish()

accuracy,▁▄▆▆▇▇▇█████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▃▆▃▇▇▇▆▆▅▇▇██▇▆▇▇
val_loss,▁▂▁▃▄▅▄▅▆▆█▇▆▇▆▆▇▇▇█
accuracy,0.99057
best_epoch,2
best_val_loss,0.32894
epoch,19
loss,0.02481
val_accuracy,0.87659
